In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install --no-index h5py

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [10]:
from tqdm import tqdm
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import cProfile
import pstats
import io

import numpy as np
from scipy.stats import norm
import scipy.stats as stats

import warnings

import logging
from abc import abstractmethod

pd.options.mode.copy_on_write = True

In [11]:
class BehavioralMetrics:
    def __init__(
        self,
        h5_path: str = None,
        dataset_name: str = None,
        verbose: bool = False,
        df: pd.DataFrame = None,
    ) -> None:
        if df is not None:
            self.behavioral_data = df
            self.verbose = verbose
            return

        self.dataset_name = dataset_name
        self.h5_path = h5_path
        self.verbose = verbose
        self.read_h5()  # read the h5 file
        self.h5_processing()  # process the h5 file

    def h5_processing(self):
        df = pd.DataFrame(
            self.h5_obj[self.dataset_name][:]
        ).T  # convert the h5 dataset to a pandas dataframe
        df.columns = [
            "image_number",
            "fix_dur_sample",
            "presentation_time_ms",
            "trial_completed",
            "target",
            "distractor",
            "correct",
            "reaction_time_ms",
            "delay_ms",
            "date",
        ]  # rename the columns
        df.columns = df.columns.astype(str)

        # df = df[df["trial_completed"] == 1]
        # for column_name in df.columns:
        #     df[column_name] = df[column_name].astype(int) # convert the columns to int

        self.behavioral_data = df

        if self.verbose:
            logging.info("h5 file processed successfully!")

    def read_h5(self):
        h5_obj = h5py.File(self.h5_path, "r")
        self.h5_obj = h5_obj

        if self.verbose:
            logging.info("h5 file read successfully!")

class BehavioralMetrics:
    def __init__(
        self,
        h5_path: str = None,
        dataset_name: str = None,
        verbose: bool = False,
        df: pd.DataFrame = None,
    ) -> None:
        if df is not None:
            self.behavioral_data = df
            self.verbose = verbose
            return

        self.dataset_name = dataset_name
        self.h5_path = h5_path
        self.verbose = verbose
        self.read_h5()  # read the h5 file
        self.h5_processing()  # process the h5 file

    def h5_processing(self):
        df = pd.DataFrame(
            self.h5_obj[self.dataset_name][:]
        ).T  # convert the h5 dataset to a pandas dataframe
        df.columns = [
            "image_number",
            "fix_dur_sample",
            "presentation_time_ms",
            "trial_completed",
            "target",
            "distractor",
            "correct",
            "reaction_time_ms",
            "delay_ms",
            "date",
        ]  # rename the columns
        df.columns = df.columns.astype(str)

        # df = df[df["trial_completed"] == 1]
        # for column_name in df.columns:
        #     df[column_name] = df[column_name].astype(int) # convert the columns to int

        self.behavioral_data = df

        if self.verbose:
            logging.info("h5 file processed successfully!")

    def read_h5(self):
        h5_obj = h5py.File(self.h5_path, "r")
        self.h5_obj = h5_obj

        if self.verbose:
            logging.info("h5 file read successfully!")

    def get_accuracy(self):
        df = self.behavioral_data.copy()
        df = df[["image_number", "target", "distractor", "correct"]]
        df.dropna(subset=["correct"], inplace=True)
        df.sort_values(by=["target", "image_number", "distractor"], inplace=True)

        accuracies = df.groupby("image_number")["correct"].mean().unstack().to_numpy()

        return (
            self.behavioral_data["correct"].dropna().mean()
        )  # calculate the accuracy of the behavioral data

    def get_standard_error(self, iteration: int = 100):
        behavior_value = self.behavioral_data["correct"].dropna().to_numpy()

        # To ensure the error bars do not go beyond 0 or 1,
        # you can use the adjusted Wald method (also known as the Agresti-Coull interval)
        # to calculate the SE for a proportion.
        # This method adjusts the proportion and the sample size slightly to account for the fact that the sampling distribution of a proportion is not symmetric,
        # especially with small sample sizes or proportions near 0 or 1.

        p_adj = (behavior_value.sum() + 2) / (len(behavior_value) + 4)
        se = np.sqrt(p_adj * (1 - p_adj) / len(behavior_value))

        print(f"{len(behavior_value) = }")
        se = np.std(behavior_value)
        return se  # calculate the standard deviation of the behavioral data

    def get_info(self):
        df = self.behavioral_data.copy()
        df = df[["image_number", "target", "distractor", "correct"]]
        df.sort_values(by=["target", "image_number", "distractor"], inplace=True)
        df.dropna(subset=["correct"], inplace=True)

        # Print the unique number of images
        print(f"unique Images: {df['image_number'].nunique()}")

        # print the unique number of targets
        print(f"unique Targets: {df['target'].nunique()}")

        # print the unique number of distractors per target
        plt.figure(figsize=(3, 2))
        df.groupby("target")["distractor"].nunique().plot(kind="bar")
        plt.title("unique Distractors per target")
        plt.grid()
        plt.yticks(range(1, 10))
        plt.show()
        # print(f"unique Distractors: {df.groupby('target')['distractor'].nunique()}")

        # plot bar plot the unique image number per target
        plt.figure(figsize=(3, 2))
        df.groupby("target")["image_number"].nunique().plot(kind="bar")
        plt.title("unique Image number per target")
        plt.grid()
        plt.show()
        # print(f"unique Image number per target: {df.groupby('target')['image_number'].nunique()}")

        # show how many trials are there for each target
        plt.figure(figsize=(7, 2))
        ax = df.groupby("target")["correct"].count().plot(kind="bar")
        plt.title("Trials per target")
        plt.grid()

        # Add text annotations to the bars
        for p in ax.patches:
            ax.annotate(
                str(p.get_height()),
                (p.get_x() + p.get_width() / 2, p.get_height()),
                ha="center",
                va="bottom",
            )

        plt.show()

        # show how many trials are there for each image
        plt.figure(figsize=(7, 2))
        # plot the histogram of the trials per image
        df.groupby("image_number")["correct"].count().plot(
            kind="hist", bins=10, rwidth=0.8, color="#0504aa"
        )
        plt.title("Histogram Trials per image")
        plt.grid()
        plt.show()

    @abstractmethod
    def calculate_behavioral_metric(self):
        raise NotImplementedError("Subclass must implement this method!")


class BehavioralMetricsI(BehavioralMetrics):
    def __init__(
        self,
        h5_path: str = None,
        dataset_name: str = None,
        verbose: bool = False,
        df: pd.DataFrame = None,
    ) -> None:
        super().__init__(h5_path, dataset_name, verbose, df)
        self.behavioral_metric = {"I1": None, "I2": None}
        self.calculate_behavioral_metric()  # calculate the behavioral metric

    def calculate_behavioral_metric(self):
        # sort base on target class
        df_delay_100_slice = self.behavioral_data[
            ["image_number", "target", "distractor", "correct"]
        ]  # slice the dataframe

        df_delay_100_slice.sort_values(
            by=["target", "image_number", "distractor"], inplace=True
        )  # sort the dataframe base on target class and image number and distractor class and inplace the matrix
        df_delay_100_slice.dropna(
            subset=["correct"], inplace=True
        )  # drop the nan values in the correct column and inplace the matrix

        # calculate the I1 behavioral metric: the mean of correct responses for each image regardless of distractor class
        b_i1 = self.calculate_bi1(
            df_delay_100_slice
        )  # calculate the I1 behavior metric

        b_i2 = self.calculate_bi2(
            df_delay_100_slice
        )  # calculate the I1 behavior metric
        self.behavioral_metric = {"B.I1": b_i1, "B.I2": b_i2}

        if self.verbose:
            logging.info("I1 behavioral metric calculated successfully!")

    def calculate_bi2(self, df):
        # Group by both 'image_number' and 'distractor', and calculate the mean of 'correct'
        grouped = df.groupby(["image_number", "distractor"])["correct"].mean()

        # Reshape the data into a list of matrices, one for each unique 'image_number'
        images = [
            group.unstack().to_numpy().ravel() for _, group in grouped.groupby(level=0)
        ]

        return np.array(images)

    def get_bi1(self):
        return self.behavioral_metric["B.I1"]

    def calculate_bi1(self, df):
        return (
            df.groupby(["image_number"])
            .agg({"correct": "mean"})
            .unstack()
            .to_numpy()
            .reshape(-1, 1)
        )

    def calculate_internal_reliability(self):
        intenal_consistency = []
        for _ in range(100):
            df = self.behavioral_data.copy()
            df = df[["image_number", "target", "distractor", "correct"]]
            # Assuming df is your pandas DataFrame
            df = df.sample(frac=1).reset_index(
                drop=True
            )  # randomize the behavioral data
            df = df.dropna(
                subset=["correct"]
            )  # drop the nan values in the correct column
            df.sort_values(by=["target", "image_number", "distractor"], inplace=True)

            split1, split2 = [], []
            for image_number in df["image_number"].unique():
                image_trials = df[df["image_number"] == image_number]

                # split the trials into two halves with equal number of trials
                # print(f"{len(image_trials) = }")
                # print(f"{len(image_trials) = }")
                # print(f"{image_trials.iloc[: len(image_trials) // 2].shape = }")
                # print(f"{image_trials.iloc[len(image_trials) // 2, :].shape = }")

                if diff := len(image_trials) % 2:
                    image_trials = image_trials[:-diff]

                # print(f"{image_trials.iloc[: len(image_trials) // 2].shape = }")
                # print(f"{image_trials.iloc[len(image_trials) // 2, :].shape = }")

                split1.append(
                    image_trials.iloc[: len(image_trials) // 2]["correct"].mean()
                )
                split2.append(
                    image_trials.iloc[len(image_trials) // 2 :]["correct"].mean()
                )

            # print(f"{len(split1) = }")
            # print(f"{len(split2) = }")

            correlation, _ = pearsonr(split1, split2)

            split_half_reliability = 2 * correlation / (1 + correlation)
            intenal_consistency.append(split_half_reliability)
            # print(f"Split-half reliability coefficient: {split_half_reliability}")

        print(f"{np.mean(intenal_consistency) = }")

    def dataframe_split_half(self):
        # randomize the behavioral data
        df = self.behavioral_data.copy()
        # Assuming df is your pandas DataFrame
        df = df.sample(frac=1).reset_index(drop=True)

        df = df[["image_number", "target", "distractor", "correct"]]
        df = df.dropna(subset=["correct"])

        # Calculate the number of trials for unique trials
        images_repetition = df.groupby("image_number").size()
        min_repetition = images_repetition.min()

        # if the minium repetition is odd, subtract 1
        if min_repetition % 2 != 0:
            min_repetition -= 1

        # Create a new DataFrame to store the equalized data
        equalized_df = []

        for image_number in df["image_number"].unique():
            image_trials = df[df["image_number"] == image_number]

            # If the number of trials is more than the minimum, sample 'min_trials' number of rows
            if len(image_trials) > min_repetition:
                image_trials = image_trials.sample(n=min_repetition)

            # Append the sampled rows to the new DataFrame
            equalized_df.append(image_trials)

        # Reset the index of the new DataFrame
        # Concatenate all sampled DataFrames into one
        equalized_df = pd.concat(equalized_df).reset_index(drop=True)

        # Define a function that splits each group into two DataFrames
        def split_group(group):
            half = len(group) // 2
            return group.iloc[:half], group.iloc[half:]

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            halves = equalized_df.groupby("image_number").apply(
                lambda x: split_group(x)
            )
        # Concatenate the first and second halves separately to form two DataFrames

        df1 = pd.concat([half[0] for half in halves])
        df2 = pd.concat([half[1] for half in halves])

        # Reset index for both DataFrames
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)

        return df1, df2

    def plot_behavioral_metric_b1(self):
        # Plot the data as a heatmap
        fig, ax = plt.subplots(figsize=(0.5, 6))
        cax = ax.imshow(self.get_bi1(), aspect="auto", cmap="Spectral")

        # Add a colorbar
        fig.colorbar(cax)

        # Remove x-axis since it's not meaningful in this context
        ax.set_xticks([])

        # Optionally, you can set the y-axis to show indices or other meaningful labels
        ax.set_yticks(
            []
        )  # or set to a range of values that are meaningful for your data

        if self.verbose:
            logging.info("I1 behavioral metric plotted successfully!")

        return cax

    def __repr__(self):
        return f"Image-based Behavioral Metric"


class BehavioralMetricsO(BehavioralMetrics):
    def __init__(self, h5_path: str, dataset_name: str, verbose: bool = False) -> None:
        super().__init__(h5_path, dataset_name, verbose)
        self.behavioral_metric = {"B.O1": None, "B.O2": None}
        self.calculate_behavioral_metric()  # calculate the behavioral metric

    def calculate_behavioral_metric(self):
        df_delay_100_slice = self.behavioral_data[
            ["image_number", "target", "distractor", "correct"]
        ]

        # Create target_unique_values dictionary more efficiently
        unique_targets = df_delay_100_slice["target"].dropna().unique().astype(int)
        target_unique_values = {
            target: [] for target in sorted(unique_targets)
        }  # Sorted the tagets to make the dictionary more readable

        # Calculate images_per_class more efficiently using groupby
        grouped = df_delay_100_slice.groupby(["target", "distractor"]).size()
        images_per_class = grouped.values

        # Populate target_unique_values dictionary
        max_length = max(images_per_class)

        for target in target_unique_values.keys():
            temp_df = df_delay_100_slice[df_delay_100_slice["target"] == target]
            for distractor in target_unique_values.keys():
                if target == distractor:
                    target_unique_values[target].append(np.full(max_length, np.nan))
                else:
                    corr_array = temp_df[temp_df["distractor"] == distractor][
                        "correct"
                    ].to_numpy()

                    # Pad the array if it's shorter than max_length
                    pad_length = max_length - len(corr_array)
                    if pad_length > 0:
                        corr_array = np.pad(
                            corr_array,
                            (0, pad_length),
                            "constant",
                            constant_values=np.nan,
                        )
                    target_unique_values[target].append(corr_array)

        # Convert target_unique_values dictionary to List
        target_unique_values_array = np.array(
            [
                np.array(target_unique_values[target])
                for target in sorted(target_unique_values.keys())
            ]
        )

        with warnings.catch_warnings():
            # RuntimeWarning: Mean of empty slice
            warnings.simplefilter("ignore", category=RuntimeWarning)
            b_o2 = np.nanmean(target_unique_values_array, axis=2)

        b_o1 = np.nanmean(target_unique_values_array, axis=(2, 1)).reshape(-1, 1)
        # calculate the O1 behavior metric
        self.behavioral_metric = {"B.O1": b_o1, "B.O2": b_o2}

    def get_bo1(self):
        return self.behavioral_metric["B.O1"]

    def get_bo2(self):
        return self.behavioral_metric["B.O2"]

    def plot_behavioral_metric_b1(self):
        # Plot the data as a heatmap
        fig, ax = plt.subplots(figsize=(0.5, 6))
        cax = ax.imshow(self.get_bo1(), aspect="auto", cmap="Spectral")

        # Add a colorbar
        fig.colorbar(cax)

        # Remove x-axis since it's not meaningful in this context
        ax.set_xticks([])

        # Optionally, you can set the y-axis to show indices or other meaningful labels
        ax.set_yticks(
            []
        )  # or set to a range of values that are meaningful for your data

        if self.verbose:
            logging.info("I1 behavioral metric plotted successfully!")

        return fig

    def plot_behavioral_metric_b2(self):
        # Plot the data as a heatmap
        fig, ax = plt.subplots(figsize=(5, 5))
        cax = ax.imshow(self.get_bo2(), aspect="auto", cmap="Spectral")

        # Add a colorbar
        fig.colorbar(cax)

        # Remove x-axis since it's not meaningful in this context
        ax.set_xticks([])

        # Optionally, you can set the y-axis to show indices or other meaningful labels
        ax.set_yticks(
            []
        )  # or set to a range of values that are meaningful for your data

        if self.verbose:
            logging.info("I1 behavioral metric plotted successfully!")

        return fig

    def __repr__(self) -> str:
        return "Object-based Behavioral Metric"

In [12]:
bm_obj =  BehavioralMetricsI(h5_path="data/wm_delay_data_v2.h5", dataset_name="delay_100", verbose=True)

In [38]:
bm_obj.behavioral_metric["B.I1"]

array([[0.9125    ],
       [0.95604396],
       [0.96470588],
       [0.8988764 ],
       [0.9494382 ],
       [0.9202454 ],
       [0.9       ],
       [0.8373494 ],
       [0.86781609],
       [0.93529412],
       [0.96341463],
       [0.87272727],
       [0.87719298],
       [0.92352941],
       [0.92771084],
       [0.87878788],
       [0.90588235],
       [0.89285714],
       [0.89820359],
       [0.84848485],
       [0.91812865],
       [0.82738095],
       [0.75739645],
       [0.86982249],
       [0.88554217],
       [0.9127907 ],
       [0.82022472],
       [0.95092025],
       [0.90643275],
       [0.93292683],
       [0.85714286],
       [0.83908046],
       [0.81547619],
       [0.86857143],
       [0.75882353],
       [0.9408284 ],
       [0.91566265],
       [0.84756098],
       [0.87261146],
       [0.91071429],
       [0.99438202],
       [0.9245283 ],
       [1.        ],
       [0.95375723],
       [0.98214286],
       [0.95238095],
       [0.98224852],
       [0.970

In [13]:
bm_obj.behavioral_data.head()

,image_number,fix_dur_sample,presentation_time_ms,trial_completed,target,distractor,correct,reaction_time_ms,delay_ms,date
0,0.0,1.0,100.0,1.0,0.0,6.0,1.0,581.000,100.0,20160417.0
1,0.0,0.0,100.0,1.0,NaN,NaN,NaN,NaN,100.0,20160418.0
2,0.0,1.0,100.0,1.0,0.0,9.0,1.0,688.000,100.0,20160419.0
3,0.0,1.0,100.0,1.0,0.0,1.0,0.0,1519.263,100.0,20160421.0
4,0.0,1.0,100.0,1.0,0.0,3.0,1.0,554.000,100.0,20160423.0


In [14]:
bm_obj.behavioral_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36641 entries, 0 to 36640
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   image_number          36641 non-null  float64
 1   fix_dur_sample        36641 non-null  float64
 2   presentation_time_ms  36641 non-null  float64
 3   trial_completed       36641 non-null  float64
 4   target                33471 non-null  float64
 5   distractor            33471 non-null  float64
 6   correct               33471 non-null  float64
 7   reaction_time_ms      33471 non-null  float64
 8   delay_ms              36641 non-null  float64
 9   date                  36641 non-null  float64
dtypes: float64(10)
memory usage: 2.8 MB


In [15]:
bm_obj.behavioral_data["image_number"].unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [36]:
bm_obj.behavioral_data.sort_values(by=["image_number"]).image_number.value_counts()

image_number
57.0     312
168.0    291
193.0    216
173.0    214
192.0    211
        ... 
172.0    170
141.0    170
155.0    169
90.0     168
136.0    167
Name: count, Length: 200, dtype: int64

In [28]:
temp = bm_obj.behavioral_data.copy()

In [37]:
len(temp[temp["image_number"] == 0])

192